In [1]:
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date

In [2]:
api = SentinelAPI('szymonoso', 'Herbata123', 'https://apihub.copernicus.eu/apihub')
# api = SentinelAPI('admin', 'password', 'https://apihub.copernicus.eu/apihub')

In [3]:
# footprint = geojson_to_wkt(read_geojson('malopolska.geojson'))

def getQuery(down_left, top_right, start_date, end_date):
    footprint = "POLYGON(("
    footprint += down_left[0] + " " + down_left[1] + ","
    footprint += down_left[0] + " " + top_right[1] + ","
    footprint += top_right[0] + " " + top_right[1] + ","
    footprint += top_right[0] + " " + down_left[1] + ","
    footprint += down_left[0] + " " + down_left[1]
    footprint += "))"

    return api.query(footprint,
                     date=(start_date, end_date),
                     platformname='Sentinel-2',
                     processinglevel='Level-2A',
                     cloudcoverpercentage=(0, 20))


In [4]:
products = getQuery(("19.792557", "50.004650"), ("20.102234", "50.107368"), date(2022, 1, 1), date(2022, 1, 31)) # Kraków from January

# examples:
# products = getQuery(("19.792557", "50.004650"), ("20.102234", "50.107368"), date(2020, 1, 1), date(2022, 1, 1))
# products = getQuery(("19.792557", "50.004650"), ("20.102234", "50.107368"), date(2000, 1, 1), "NOW")
# products = getQuery(("19.792557", "50.004650"), ("20.102234", "50.107368"), "NOW-5DAYS", "NOW")

In [5]:
products_df = api.to_dataframe(products)
products_df

,title,link,link_alternative,link_icon,summary,ondemand,generationdate,beginposition,endposition,ingestiondate,...,s2datatakeid,producttype,platformidentifier,orbitdirection,platformserialidentifier,processinglevel,datastripidentifier,granuleidentifier,identifier,uuid
81b4de28-ebcf-4bdf-9a21-d19a901693e7,S2A_MSIL2A_20220129T094241_N0400_R036_T34UDA_2...,https://apihub.copernicus.eu/apihub/odata/v1/P...,https://apihub.copernicus.eu/apihub/odata/v1/P...,https://apihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2022-01-29T09:42:41.024Z, Instrument: MS...",false,2022-01-29 13:13:46,2022-01-29 09:42:41.024,2022-01-29 09:42:41.024,2022-01-29 16:43:13.331,...,GS2A_20220129T094241_034496_N04.00,S2MSI2A,2015-028A,DESCENDING,Sentinel-2A,Level-2A,S2A_OPER_MSI_L2A_DS_VGS2_20220129T131346_S2022...,S2A_OPER_MSI_L2A_TL_VGS2_20220129T131346_A0344...,S2A_MSIL2A_20220129T094241_N0400_R036_T34UDA_2...,81b4de28-ebcf-4bdf-9a21-d19a901693e7
3d54303b-cc4e-437a-9878-b2da193460a4,S2B_MSIL2A_20220124T094159_N0301_R036_T34UDA_2...,https://apihub.copernicus.eu/apihub/odata/v1/P...,https://apihub.copernicus.eu/apihub/odata/v1/P...,https://apihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2022-01-24T09:41:59.024Z, Instrument: MS...",false,2022-01-24 11:17:02,2022-01-24 09:41:59.024,2022-01-24 09:41:59.024,2022-01-24 14:14:32.858,...,GS2B_20220124T094159_025516_N03.01,S2MSI2A,2017-013A,DESCENDING,Sentinel-2B,Level-2A,S2B_OPER_MSI_L2A_DS_VGS4_20220124T111702_S2022...,S2B_OPER_MSI_L2A_TL_VGS4_20220124T111702_A0255...,S2B_MSIL2A_20220124T094159_N0301_R036_T34UDA_2...,3d54303b-cc4e-437a-9878-b2da193460a4
d876456b-5a8d-486e-a656-736beae4277a,S2A_MSIL2A_20220109T094401_N0301_R036_T34UDA_2...,https://apihub.copernicus.eu/apihub/odata/v1/P...,https://apihub.copernicus.eu/apihub/odata/v1/P...,https://apihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2022-01-09T09:44:01.024Z, Instrument: MS...",false,2022-01-09 12:12:35,2022-01-09 09:44:01.024,2022-01-09 09:44:01.024,2022-01-09 14:40:49.921,...,GS2A_20220109T094401_034210_N03.01,S2MSI2A,2015-028A,DESCENDING,Sentinel-2A,Level-2A,S2A_OPER_MSI_L2A_DS_VGS4_20220109T121235_S2022...,S2A_OPER_MSI_L2A_TL_VGS4_20220109T121235_A0342...,S2A_MSIL2A_20220109T094401_N0301_R036_T34UDA_2...,d876456b-5a8d-486e-a656-736beae4277a
5adace80-1c19-426e-81e5-96fb122a6f29,S2B_MSIL2A_20220107T095309_N0301_R079_T34UDA_2...,https://apihub.copernicus.eu/apihub/odata/v1/P...,https://apihub.copernicus.eu/apihub/odata/v1/P...,https://apihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2022-01-07T09:53:09.024Z, Instrument: MS...",false,2022-01-07 12:08:20,2022-01-07 09:53:09.024,2022-01-07 09:53:09.024,2022-01-07 16:49:41.644,...,GS2B_20220107T095309_025273_N03.01,S2MSI2A,2017-013A,DESCENDING,Sentinel-2B,Level-2A,S2B_OPER_MSI_L2A_DS_VGS4_20220107T120820_S2022...,S2B_OPER_MSI_L2A_TL_VGS4_20220107T120820_A0252...,S2B_MSIL2A_20220107T095309_N0301_R079_T34UDA_2...,5adace80-1c19-426e-81e5-96fb122a6f29


In [6]:
# download all results from the search
# api.download_all(products)